In [1]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

In [2]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [3]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [4]:
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = datagen.flow_from_directory('images/train/',
        class_mode='binary', batch_size=32, target_size=(200, 200))
    test_it = datagen.flow_from_directory('images/test/',
        class_mode='binary', batch_size=32, target_size=(200, 200))
    
    start_time = time.time()

    # fit model
    history = model.fit(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=15, verbose=0)
    end_time = time.time()
    # evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
    training_time = end_time - start_time
    print("Training Time: %.2f seconds" % training_time)

    print("Training Loss:")
    print(history.history['loss'])
    print("Training Accuracy:")
    print(history.history['accuracy'])
    print("Testing Accuracy:")
    print('> %.3f' % (acc * 100.0))
    model.summary()
    summarize_diagnostics(history)

In [5]:
run_test_harness() 


Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


/Users/rutwikmore/anaconda3/lib/python3.11/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Training Time: 88.96 seconds
Training Loss:
[10.15640640258789, 0.6982228755950928, 0.691074550151825, 0.6712760925292969, 0.6838899850845337, 0.6669395565986633, 0.6345689296722412, 0.6354802846908569, 0.6781090497970581, 0.6295895576477051, 0.5967365503311157, 0.5963019728660583, 0.4879414141178131, 0.5628796815872192, 0.4305707514286041]
Training Accuracy:
[0.4437499940395355, 0.48124998807907104, 0.53125, 0.5687500238418579, 0.574999988079071, 0.5375000238418579, 0.831250011920929, 0.7437499761581421, 0.518750011920929, 0.71875, 0.75, 0.7562500238418579, 0.831250011920929, 0.699999988079071, 0.800000011920929]
Testing Accuracy:
> 72.500
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 32)      0         
 